<a href="https://colab.research.google.com/github/kasev/PIA/blob/master/scripts/2_EXPLORATIONS.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [7]:
### PREREQUISTIES
### (many used only in one notebook...)

import os
import pandas as pd
import numpy as np
import logging ### to monitor the code
from bs4 import BeautifulSoup
import pickle
import math
import random
import sys
import csv
import unicodedata
import requests
import re

import nltk
from nltk.collocations import *

import networkx as nx

import matplotlib.pyplot as plt
import seaborn as sns

from scipy import stats

import xml.etree.cElementTree as ET
from urllib.request import urlopen

import gspread
from gspread_dataframe import get_as_dataframe, set_with_dataframe

!pip install sddk
import sddk

In [2]:
!pip install anda
from anda import gr ### the import takes substantial time, since it import a +600MB file containing ancient Greek dictionary

     |████████████████████████████████| 21.3MB 1.3MB/s 


In [8]:
### not neccessary for reading the data, just for exporting them to sciencedata.dk
conf = sddk.configure_session_and_url("SDAM_root", "648597@au.dk")

sciencedata.dk username (format '123456@au.dk'): 648597@au.dk
sciencedata.dk password: ··········
connection with shared folder established with you as its owner
endpoint variable has been configured to: https://sciencedata.dk/files/SDAM_root/


In [0]:
### authorize google sheets
from google.colab import auth
auth.authenticate_user()
from oauth2client.client import GoogleCredentials
gc = gspread.authorize(GoogleCredentials.get_application_default())
# establish connection with particular sheet by its url:

PIA_data = gc.open_by_url("https://docs.google.com/spreadsheets/d/1KxOx7Be9fj3lDcEPgQhQ-Iqcn9p367-MMD6RMXe8rks/edit?usp=sharing")
PIA_overview = gc.open_by_url("https://docs.google.com/spreadsheets/d/1e94wyelg6dftQ4zxbq1xvwxWAI-BhcYXtclDW-YTnrw/edit?usp=sharing")

In [16]:
### our data are located within a publicly available folder on sciencedata.dk

publicfolder = "31b393e2afe1ee96ce81869c7efe18cb"
c_aristotelicum = sddk.read_file("c_aristotelicum.json", "df", "31b393e2afe1ee96ce81869c7efe18cb")
c_hippocraticum = sddk.read_file("c_hippocraticum.json", "df", "31b393e2afe1ee96ce81869c7efe18cb")

this file is located in a public folder
this file is located in a public folder


In [17]:
c_aristotelicum.head(5)

,filename,author,title,string,author_id,doc_id,raw_date,date_avr,date_probs,date_manual,provenience,wordcount,lemmata,lemmatized_sentences,lemmata_wordcount,num_of_sents
678,tlg0086.tlg001.1st1K-grc2.xml,Aristotle,Aristotelis Analytica Priora et Posteriora; Ar...,ΑΝΑΛΥΤΙΚΩΝ ΠΡΟΤΕΡΩΝ Α. Πρῶτον εἰπεῖν περὶ τί κ...,tlg0086,tlg0086.tlg001,4 B.C.,-3.5,{'-3.5': 1},-3.5,pagan,59772,"[αναλυτικων, προτερων, πρῶτον, εἶπον, σκέψις, ...","[[αναλυτικων, προτερων], [πρῶτον, εἶπον, σκέψι...",21150,3645
679,tlg0086.tlg002.1st1K-grc2.xml,Aristotle,De anima; Aritoteles De anima,ΠEP1 ΨΥΧΗΣ Α Τῶν καλῶν καὶ τιμίων τὴν εἴδησιν ...,tlg0086,tlg0086.tlg002,4 B.C.,-3.5,{'-3.5': 1},-3.5,pagan,20988,"[ψυχης, κᾶλον, τίμιος, εἶδος, μαλλός, ἕτερος, ...","[[ψυχης, κᾶλον, τίμιος, εἶδος, μαλλός, ἕτερος,...",8893,1306
681,tlg0086.tlg005.1st1K-grc1.xml,Aristotle,De caelo; Aristoteles De coelo et De generatio...,ΠEPI ΟΥΡΑΝΟΥ A. Ἡ περὶ φύσεως ἐπιστήμη σχεδὸν ...,tlg0086,tlg0086.tlg005,4 B.C.,-3.5,{'-3.5': 1},-3.5,pagan,31395,"[ουρανου, φύσις, ἐπιστήμη, πλεῖστος, φαίνω, σῶ...","[[ουρανου], [φύσις, ἐπιστήμη, πλεῖστος, φαίνω,...",12915,1986
682,tlg0086.tlg006.1st1K-grc1.xml,Aristotle,Categoriae; Aristotelis Opera,ΚΑΤΗΓΟΡΙΑΙ. ὉΜΩΝΥΜΑ λέγεται ὧν ὄνομα μόνον κοι...,tlg0086,tlg0086.tlg006,4 B.C.,-3.5,{'-3.5': 1},-3.5,pagan,10317,"[κατηγοριαι, ὁμωνυμα, λέγω, ὄνομα, μόνος, κοιν...","[[κατηγοριαι], [ὁμωνυμα, λέγω, ὄνομα, μόνος, κ...",4372,659
683,tlg0086.tlg008.1st1K-grc1.xml,Aristotle,De divinatione per somnum; Aristotelis Opera,ΠΕΡΙ ΕΝΥΠΝΙΩΝ ΚΑΙ ΤΗΣ ΚΑΘ’ ΥΠΝΟΝ ΜΑΝΤΙΚΗΣ. Περ...,tlg0086,tlg0086.tlg008,4 B.C.,-3.5,{'-3.5': 1},-3.5,pagan,1207,"[περι, ενυπνιων, και, της, καθ, υπνον, μαντικη...","[[περι, ενυπνιων, και, της, καθ, υπνον, μαντικ...",533,69


# Exploring Nicomachean Ethics

In [18]:
EN_df = c_aristotelicum[c_aristotelicum["doc_id"]=="tlg0086.tlg010"]
EN_df

,filename,author,title,string,author_id,doc_id,raw_date,date_avr,date_probs,date_manual,provenience,wordcount,lemmata,lemmatized_sentences,lemmata_wordcount,num_of_sents
685,tlg0086.tlg010.perseus-grc1.xml,Aristotle,Nicomachean Ethics; Machine readable text; ed....,"βοοκ 1 πᾶσα τέχνη καὶ πᾶσα μέθοδος, ὁμοίως δὲ ...",tlg0086,tlg0086.tlg010,4 B.C.,-3.5,{'-3.5': 1},-3.5,pagan,56638,"[βοοκ, πᾶς, τέχνη, πᾶς, μέθοδος, ὅμοιος, πρᾶξί...","[[βοοκ, πᾶς, τέχνη, πᾶς, μέθοδος, ὅμοιος, πρᾶξ...",23811,3897


In [19]:
EN_df["string"].tolist()[0][:1000]

'βοοκ 1 πᾶσα τέχνη καὶ πᾶσα μέθοδος, ὁμοίως δὲ πρᾶξίς τε καὶ προαίρεσις, ἀγαθοῦ τινὸς ἐφίεσθαι δοκεῖ· διὸ καλῶς ἀπεφήναντο τἀγαθόν, οὗ πάντʼ ἐφίεται. διαφορὰ δέ τις φαίνεται τῶν τελῶν· τὰ μὲν γάρ εἰσιν ἐνέργειαι, τὰ δὲ παρʼ αὐτὰς ἔργα τινά. ὧν δʼ εἰσὶ τέλη τινὰ παρὰ τὰς πράξεις, ἐν τούτοις βελτίω πέφυκε τῶν ἐνεργειῶν τὰ ἔργα. πολλῶν δὲ πράξεων οὐσῶν καὶ τεχνῶν καὶ ἐπιστημῶν πολλὰ γίνεται καὶ τὰ τέλη· ἰατρικῆς μὲν γὰρ ὑγίεια, ναυπηγικῆς δὲ πλοῖον, στρατηγικῆς δὲ νίκη, οἰκονομικῆς δὲ πλοῦτος. ὅσαι δʼ εἰσὶ τῶν τοιούτων ὑπὸ μίαν τινὰ δύναμιν, καθάπερ ὑπὸ τὴν ἱππικὴν χαλινοποιικὴ καὶ ὅσαι ἄλλαι τῶν ἱππικῶν ὀργάνων εἰσίν, αὕτη δὲ καὶ πᾶσα πολεμικὴ πρᾶξις ὑπὸ τὴν στρατηγικήν, κατὰ τὸν αὐτὸν δὴ τρόπον ἄλλαι ὑφʼ ἑτέρας· ἐν ἁπάσαις δὲ τὰ τῶν ἀρχιτεκτονικῶν τέλη πάντων ἐστὶν αἱρετώτερα τῶν ὑπʼ αὐτά· τούτων γὰρ χάριν κἀκεῖνα διώκεται. διαφέρει δʼ οὐδὲν τὰς ἐνεργείας αὐτὰς εἶναι τὰ τέλη τῶν πράξεων ἢ παρὰ ταύτας ἄλλο τι, καθάπερ ἐπὶ τῶν λεχθεισῶν ἐπιστημῶν. εἰ δή τι τέλος ἐστὶ τῶν πρακτῶν ὃ διʼ αὑτ

In [20]:
print(EN_df["lemmatized_sentences"].tolist()[0][:10])

[['βοοκ', 'πᾶς', 'τέχνη', 'πᾶς', 'μέθοδος', 'ὅμοιος', 'πρᾶξίς', 'προαίρεσις', 'ἀγαθός', 'ἐφίημι', 'δοκέω'], ['κάλως', 'ἀποφαίνω', 'ἀγαθός', 'ἵημι', 'πάντʼ', 'ἐφίημι'], ['διαφορά', 'φαίνω', 'τέλος'], ['εἶμι', 'ἐνέργεια', 'παρʼ', 'ἔργον'], ['εἶμι', 'τέλος', 'πρᾶξις', 'βελτίων', 'φύω', 'ἐνέργεια', 'ἔργον'], ['πολύς', 'πρᾶξις', 'τέχνη', 'ἐπιστήμη', 'πολύς', 'γίγνομαι', 'τέλος'], ['ἰατρικός', 'ὑγίεια', 'ναυπηγικός', 'πλοῖον', 'στρατηγικός', 'νίκη', 'οἰκονομικός', 'πλοῦτος'], ['ὅσος', 'εἶμι', 'μίαν', 'δύναμις', 'ἱππικην', 'χαλινοποιική', 'ὅσος', 'ἱππικός', 'ὄργανον', 'εἶμι', 'πᾶς', 'πολεμικός', 'πρᾶξις', 'στρατηγικός', 'τρόπος', 'ὑφʼ', 'ἕτερος'], ['ἅπας', 'ἀρχιτεκτονικῶν', 'τέλος', 'πᾶς', 'αἱρετός', 'ὑπʼ'], ['χάρις', 'ἐκεῖνος', 'διώκω']]


In [21]:
print(EN_df["lemmata"].tolist()[0])

['βοοκ', 'πᾶς', 'τέχνη', 'πᾶς', 'μέθοδος', 'ὅμοιος', 'πρᾶξίς', 'προαίρεσις', 'ἀγαθός', 'ἐφίημι', 'δοκέω', 'κάλως', 'ἀποφαίνω', 'ἀγαθός', 'ἵημι', 'πάντʼ', 'ἐφίημι', 'διαφορά', 'φαίνω', 'τέλος', 'εἶμι', 'ἐνέργεια', 'παρʼ', 'ἔργον', 'εἶμι', 'τέλος', 'πρᾶξις', 'βελτίων', 'φύω', 'ἐνέργεια', 'ἔργον', 'πολύς', 'πρᾶξις', 'τέχνη', 'ἐπιστήμη', 'πολύς', 'γίγνομαι', 'τέλος', 'ἰατρικός', 'ὑγίεια', 'ναυπηγικός', 'πλοῖον', 'στρατηγικός', 'νίκη', 'οἰκονομικός', 'πλοῦτος', 'ὅσος', 'εἶμι', 'μίαν', 'δύναμις', 'ἱππικην', 'χαλινοποιική', 'ὅσος', 'ἱππικός', 'ὄργανον', 'εἶμι', 'πᾶς', 'πολεμικός', 'πρᾶξις', 'στρατηγικός', 'τρόπος', 'ὑφʼ', 'ἕτερος', 'ἅπας', 'ἀρχιτεκτονικῶν', 'τέλος', 'πᾶς', 'αἱρετός', 'ὑπʼ', 'χάρις', 'ἐκεῖνος', 'διώκω', 'διαφέρω', 'ἐνέργεια', 'τέλος', 'πρᾶξις', 'ἐπιστήμη', 'τέλος', 'πρακτός', 'διʼ', 'βούλομαι', 'πᾶς', 'διʼ', 'ἕτερος', 'αἱρέω', 'ʽπρόεισι', 'γʼ', 'ἤπειρος', 'ὥστʼ', 'κενός', 'μάταιος', 'ὄρεξις', 'Δῆλος', 'τοῦτʼ', 'ἐάω', 'ἀγαθός', 'ἄριστον', 'ἆρʼ', 'βιός', 'γνῶσις', 'μέγας', 'ἔχις

In [28]:
print(EN_df["lemmata"].tolist()[0][:10])

['βοοκ', 'πᾶς', 'τέχνη', 'πᾶς', 'μέθοδος', 'ὅμοιος', 'πρᾶξίς', 'προαίρεσις', 'ἀγαθός', 'ἐφίημι']


In [29]:
gr.return_all_unique_lemmata("πρᾶξίς")

'πρᾶξίς'

In [24]:
gr.lemma_translator("πρᾶξίς")

''

# Key terms

In [0]:
### produce a list of all words from the authors
aristotle_list = []
for list_element in c_aristotelicum["lemmata"].tolist():
  aristotle_list.extend(list_element)
hippocrates_list = []
for list_element in c_hippocraticum["lemmata"].tolist():
  hippocrates_list.extend(list_element)

In [31]:
keyterm_patterns = ["λ[υ|ύ]π.+", "(ἀ|ἄ)λγ.+", "ὀδ[ύ|υ]ν.", "π[ό|ο]ν[ο|ό].+"]
matches = []
for pattern in keyterm_patterns:
  r = re.compile(pattern)
  matches.extend([(pattern, match, gr.lemma_translator(match)) for match in list(filter(r.match, list(set(aristotle_list + hippocrates_list))))])
matches_df = pd.DataFrame(matches, columns=["pattern", "found", "translation"])
matches_df

,pattern,found,translation
0,λ[υ|ύ]π.+,λυπηρός,"painful, distressing"
1,λ[υ|ύ]π.+,λυπούμενός,
2,λ[υ|ύ]π.+,λυπεόντων,
3,λ[υ|ύ]π.+,λυπέωσι,
4,λ[υ|ύ]π.+,λύπαςμεσότης,
...,...,...,...
91,π[ό|ο]ν[ο|ό].+,πονοῦντιτοῦτό,
92,π[ό|ο]ν[ο|ό].+,πόνοι,
93,π[ό|ο]ν[ο|ό].+,πονοῦσι,
94,π[ό|ο]ν[ο|ό].+,πονοῦσαί,


In [0]:
keyterm_patterns = ["λ[υ|ύ]π.+", "(ἀ|ἄ)λγ.+", "ὀδ[ύ|υ]ν.", "π[ό|ο]ν[ο|ό].+"]
matches = []
for pattern in keyterm_patterns:
  r = re.compile(pattern)
  matches.extend([(pattern, match, gr.lemma_translator(match)) for match in list(filter(r.match, list(set(aristotle_list + hippocrates_list))))])
matches_df = pd.DataFrame(matches, columns=["pattern", "found", "translation"])
matches_df

,pattern,found,translation
0,λ[υ|ύ]π.+,λυπέοιτο,
1,λ[υ|ύ]π.+,λύπη,pain of body
2,λ[υ|ύ]π.+,λυπέοντος,
3,λ[υ|ύ]π.+,λυπή,
4,λ[υ|ύ]π.+,λυπέεται,
...,...,...,...
90,π[ό|ο]ν[ο|ό].+,πόνοισί,
91,π[ό|ο]ν[ο|ό].+,πονοῦσαί,
92,π[ό|ο]ν[ο|ό].+,πονοῦσι,
93,π[ό|ο]ν[ο|ό].+,πονοῦσαι,


In [0]:
set_with_dataframe(PIA_overview.add_worksheet("matches_20200521", rows=1, cols=1), matches_df)

In [0]:
patterns_dict

{'(ἀ|ἄ)λγ.+': ['ἄλγος',
  'ἄλγλος',
  'ἀλγεινός',
  'ἀλγηδὼν',
  'ἀλγηδών',
  'ἀλγέω',
  'ἀλγήσωσιν',
  'ἄλγημα',
  'ἀλγήσῃ'],
 'λ[υ|ύ]π.+': ['λύπη',
  'λυπή',
  'λυπούμενός',
  'λύπαςοὐ',
  'λυποδίαν',
  'λύπαςμεσότης',
  'λυπεῖταἰ',
  'λυπεῖταί',
  'λυπρᾷ',
  'λυπητικός',
  'λυπέω',
  'λυπηθῆ',
  'λυπηρός',
  'λυπηθήσοιτʼ'],
 'π[ό|ο]ν.+': ['πονηρολογίαν',
  'πονηροὺς',
  'πονπροστιθέασιν',
  'πόντες',
  'πόντῳ',
  'πονέω',
  'πόντου',
  'πονοῦσί',
  'πόνος',
  'πόντος',
  'ποντίλος',
  'πονῶσι',
  'πονηρόφιλος',
  'πονηρία',
  'πόντιος',
  'πονηρεύομαι',
  'ποντιάς',
  'ποντίζω',
  'πόντῳἔτι',
  'πονητικά',
  'πονεῖντῆς',
  'ποντικός',
  'πόντο',
  'ποντικοί',
  'πονοῦσαί',
  'πονηρός',
  'πονοῦσι',
  'πονεῖ',
  'ποντ',
  'πονοῦσαι',
  'πονοῦντιτοῦτό',
  'πόντον'],
 'ὀδ[ύ|υ]ν.+': ['ὀδυνηρός', 'ὀδύνατον', 'ὀδυνάω']}

In [0]:
def replacer(pattern, , product, word):
  return re.sub(pattern, pattern, word)

In [0]:
def replacer_word_list(pattern, word_list):
  return [replacer(pattern, word) for word in word_list]

In [0]:
### testing
word_list_test = ['βοοκ', 'πᾶς', "ἄλγτέχνη",'τέχνη', 'πᾶς', 'μέθοδος', 'ὅμοιος', "λύπη",'πρᾶξίς', 'προαίρεσις', 'ἀγαθός', 'ἐφίημι']
replacer_word_list("(ἀ|ἄ)λγ.+", word_list_test)

['βοοκ',
 'πᾶς',
 '(ἀ|ἄ)λγ.+',
 'τέχνη',
 'πᾶς',
 'μέθοδος',
 'ὅμοιος',
 'λύπη',
 'πρᾶξίς',
 'προαίρεσις',
 'ἀγαθός',
 'ἐφίημι']

In [0]:
### application on  whole corpora
for pattern in keyterms_new:
  c_hippocraticum["lemmata_repl"] = c_hippocraticum.apply(lambda row: replacer_word_list(pattern, row["lemmata"]), axis=1)
  c_aristotelicum["lemmata_repl"] = c_aristotelicum.apply(lambda row: replacer_word_list(pattern, row["lemmata"]), axis=1)

In [0]:
one_merged_list = []
for list_element in c_aristotelicum["lemmata_repl"].tolist():
  one_merged_list.extend(list_element)

In [0]:
len(one_merged_list)

324234

In [0]:
variants = []
for word in list(set(one_merged_list)):
  if word.startswith("ἄλγ"):
    variants.append(word)
  if word.startswith("ἀλγ"):
    variants.append(word)
print(variants)

['ἄλγλος', 'ἄλγος', 'ἄλγημα', 'ἀλγήσωσιν', 'ἀλγήσῃ', 'ἀλγέω', 'ἀλγηδὼν', 'ἀλγηδών', 'ἀλγεινός']


In [0]:
replacer_word_list("(ἀ|ἄ)λγ.+", variants)

['(ἀ|ἄ)λγ.+',
 '(ἀ|ἄ)λγ.+',
 '(ἀ|ἄ)λγ.+',
 '(ἀ|ἄ)λγ.+',
 '(ἀ|ἄ)λγ.+',
 '(ἀ|ἄ)λγ.+',
 '(ἀ|ἄ)λγ.+',
 '(ἀ|ἄ)λγ.+',
 '(ἀ|ἄ)λγ.+']

In [0]:
### application on  whole corpora
for pattern in keyterms_new:
  c_hippocraticum["lemmata_repl"] = c_hippocraticum.apply(lambda row: replacer_word_list(pattern, row["lemmata"]), axis=1)
  c_aristotelicum["lemmata_repl"] = c_aristotelicum.apply(lambda row: replacer_word_list(pattern, row["lemmata"]), axis=1)

corpus_term_tuples = [(c_hippocraticum, keyterms_new[0]), (c_hippocraticum, keyterms_new[1]),(c_hippocraticum, keyterms_new[2]), (c_hippocraticum, keyterms_new[3]), (c_aristotelicum, keyterms_new[0]), (c_aristotelicum, keyterms_new[1]), (c_aristotelicum, keyterms_new[2]), (c_aristotelicum, keyterms_new[3])]
overview = []
for corpus_term_tuple in corpus_term_tuples:
  author = corpus_term_tuple[0]["author"].tolist()[0]
  num_of_documents = len(corpus_term_tuple[0])
  length = corpus_term_tuple[0]["wordcount"].sum()
  list_of_lists = corpus_term_tuple[0]["lemmata_repl"].tolist()
  one_merged_list = []
  for list_element in list_of_lists:
    one_merged_list.extend(list_element)
  term_count = one_merged_list.count(corpus_term_tuple[1])
  term_TF = term_count/ length
  overview.append([author, str(num_of_documents) + " docs", str(length) + " words", str(corpus_term_tuple[1]) + " (N=" + str(term_count) + ")"])
pd.DataFrame(overview)

,0,1,2,3
0,Hippocrates,52 docs,334392 words,λ[υ|ύ]π.+ (N=0)
1,Hippocrates,52 docs,334392 words,(ἀ|ἄ)λγ.+ (N=0)
2,Hippocrates,52 docs,334392 words,ὀδ[ύ|υ]ν.+ (N=0)
3,Hippocrates,52 docs,334392 words,π[ό|ο]ν.+ (N=816)
4,Aristotle,27 docs,803085 words,λ[υ|ύ]π.+ (N=0)
5,Aristotle,27 docs,803085 words,(ἀ|ἄ)λγ.+ (N=0)
6,Aristotle,27 docs,803085 words,ὀδ[ύ|υ]ν.+ (N=0)
7,Aristotle,27 docs,803085 words,π[ό|ο]ν.+ (N=206)
